# Proof of Concept

The objective of this notebook is to develop a proof of concept:
- Load tweets from twarc-extracted `jsonl` files.
- Parse tweets and use `Vader` to add sentiment.
- Show a brief example analysis.

## 0. Load Env

In [40]:
import os
import json
import pandas as pd
from typing import List, Dict, Tuple, Any, Callable, Union
from functools import reduce
from operator import getitem
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)
from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pyrol\AppData\Roaming\nltk_data...


In [23]:
#RAW_DATA_DIR = "/Users/pyrol/code/gym-vendor-analysis/data/"
RAW_DATA_DIR = "../data/"

In [24]:
TWEET_DICT_KEY_MAPPING = [
    (("url",), "url",),
    (("content",), "content",),
    (("id",), "tweet_id",),
    (("user", "id",), "user_id",),
    (("user", "username",), "user_name",),
    (("date",), "datetime",),
]

In [25]:
def nested_dictionary_get(d: Dict, key: Tuple):
    return reduce(getitem, key, d)


def parse_and_reduce_tweet(tweet: Dict) -> Dict[str, Any]:
    """
    Parses a tweet (represented as a dictionary), reducing the tweet to the schema defined
    in `keys_set`
    """
    new_dict = {new_name: nested_dictionary_get(d=tweet, key=nested_key)
                for nested_key, new_name in TWEET_DICT_KEY_MAPPING}
    return new_dict

In [26]:
def load_and_parse_twarc_file_to_list_of_tweets(filename: str,
                                                include_filename_in_result=True,
                                                raw_data_dir: str = RAW_DATA_DIR
                                               ) -> List:
    path = os.path.join(raw_data_dir, f"{filename}.jsonl")
    print(f"Loading data from {path}.")
    with open(path, 'r') as f:
        if include_filename_in_result:
            tweet_list = [{**parse_and_reduce_tweet(json.loads(line)), **{"vendor": filename}} for line in f]
        else:
            tweet_list = [parse_and_reduce_tweet(json.loads(line)) for line in f]
    return tweet_list

In [27]:
def load_twarc_json_file_to_dataframe(filename: str) -> pd.DataFrame:
    tweets_list = load_and_parse_twarc_file_to_list_of_tweets(filename=filename)
    tweet_df = pd.DataFrame.from_records(tweets_list)
    return tweet_df


def load_twarc_json_files_to_dataframe(filenames: List[str]) -> pd.DataFrame:
    tweet_dfs = [load_twarc_json_file_to_dataframe(filename) for filename in filenames]
    tweet_df = pd.concat(tweet_dfs)
    return tweet_df

In [28]:
def save_tweets_df(df, path):
    print(f"Saving data to {path}.")
    df.to_parquet(path)

In [29]:
def load_tweets_df(path):
    print(f"Loading data from {path}.")
    return pd.read_parquet(path)

## 1. Load Tweets Data, Enrich with Sentiment, and Export

#### Load tweets to a dataframe

In [30]:
gym_vendors = [
    "Arsenal_Strngth",
    "BeTitanFit",
    "BodySolidFit",
    "BodycraftFit",
    "CAPBarbell84",
    "HOISTFitness",
    "HammerStrength",
    "LifeFitness",
    "PRxPerformance",
    "Powertec",
    "RepFitnessEquip",
    "RogueFitness",
    "Sorinex",
    "TorqueFitness",
    "bellsofsteel",
    "concept2",
    "cybex",
    "elitefts",
    "force_usa",
    "onepeloton"
]

In [31]:
# tweet_df = load_twarc_json_file_to_dataframe(path=paths[0], parse_tweet_fxn=parse_and_reduce_tweet)
tweet_df = load_twarc_json_files_to_dataframe(filenames=gym_vendors)

Loading data from ../data/Arsenal_Strngth.jsonl.
Loading data from ../data/BeTitanFit.jsonl.
Loading data from ../data/BodySolidFit.jsonl.
Loading data from ../data/BodycraftFit.jsonl.
Loading data from ../data/CAPBarbell84.jsonl.
Loading data from ../data/HOISTFitness.jsonl.
Loading data from ../data/HammerStrength.jsonl.
Loading data from ../data/LifeFitness.jsonl.
Loading data from ../data/PRxPerformance.jsonl.
Loading data from ../data/Powertec.jsonl.
Loading data from ../data/RepFitnessEquip.jsonl.
Loading data from ../data/RogueFitness.jsonl.
Loading data from ../data/Sorinex.jsonl.
Loading data from ../data/TorqueFitness.jsonl.
Loading data from ../data/bellsofsteel.jsonl.
Loading data from ../data/concept2.jsonl.
Loading data from ../data/cybex.jsonl.
Loading data from ../data/elitefts.jsonl.
Loading data from ../data/force_usa.jsonl.
Loading data from ../data/onepeloton.jsonl.


#### Validate Data

In [32]:
tweet_df.head().T

,0,1,2,3,4
url,https://twitter.com/Ayoobi77/status/1567340537333837824,https://twitter.com/Arsenal_Strngth/status/1567145753360977922,https://twitter.com/Arsenal_Strngth/status/1567144502334308354,https://twitter.com/Ayoobi77/status/1566312069196124166,https://twitter.com/Ayoobi77/status/1566311005038297089
content,@Arsenal_Strngth The true face of sports engineering 🙂\nArsenal Strength,Shipping soon: black on black Reloaded Iso Lat Pull down. https://t.co/GgnZl99ZhI,@Ayoobi77 @Collin_Abel10 @EaglevilleFB @CoachFWalker Thank you! ⚔️,@Collin_Abel10 @Arsenal_Strngth @EaglevilleFB @CoachFWalker The best machine,@Arsenal_Strngth Excellent products from Arsenal Strength 👍👍👍
tweet_id,1567340537333837824,1567145753360977922,1567144502334308354,1566312069196124166,1566311005038297089
user_id,787689201231888384,2942005261,2942005261,787689201231888384,787689201231888384
user_name,Ayoobi77,Arsenal_Strngth,Arsenal_Strngth,Ayoobi77,Ayoobi77
datetime,2022-09-07T02:34:38+00:00,2022-09-06T13:40:38+00:00,2022-09-06T13:35:40+00:00,2022-09-04T06:27:52+00:00,2022-09-04T06:23:38+00:00
vendor,Arsenal_Strngth,Arsenal_Strngth,Arsenal_Strngth,Arsenal_Strngth,Arsenal_Strngth


In [33]:
tweet_df.shape

(832517, 7)

In [34]:
tweet_df.isnull().mean()

url          0.0
content      0.0
tweet_id     0.0
user_id      0.0
user_name    0.0
datetime     0.0
vendor       0.0
dtype: float64

In [35]:
tweet_df.dtypes

url          object
content      object
tweet_id      int64
user_id       int64
user_name    object
datetime     object
vendor       object
dtype: object

#### Preprocess data

##### datetime field

In [36]:
tweet_df["pd_datetime"] = pd.to_datetime(tweet_df["datetime"])

In [37]:
tweet_df.dtypes

url                         object
content                     object
tweet_id                     int64
user_id                      int64
user_name                   object
datetime                    object
vendor                      object
pd_datetime    datetime64[ns, UTC]
dtype: object

#### Create Sample dataframe for testing

In [38]:
tweet_df_sample = tweet_df.sample(1000)

#### Enrich Tweets with sentiment

From [Vader docs](https://vadersentiment.readthedocs.io/en/latest/pages/about_the_scoring.html):  
> The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a ‘normalized, weighted composite score’ is accurate.

In [41]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
tweet_df["sentiment"] = tweet_df["content"].progress_apply(lambda x: analyzer.polarity_scores(x)["compound"])

 26%|██████████████████▍                                                     | 213752/832517 [05:55<04:42, 2187.42it/s]

In [ ]:
tweet_df.head().T

#### Validate sentiment

In [ ]:
tweet_df[["content", "sentiment"]].head(20)

Sentiment seems reasonable.

#### Export tweets df

In [ ]:
path = "../data/tweet_df.parquet"
save_tweets_df(tweet_df, path)

## 2. Analyze

In [ ]:
path = "../data/tweet_df.parquet"
tweet_df = load_tweets_df(path)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15, 15))
for plt_index, val, aggfunc, chart_name, chart_ylabel in [
    (0, "tweet_id", "count", "Total Tweet Volume per Month", "Count Tweets"),
    (1, "sentiment", "sum", "Total Tweet Sentiment per Month", "Sum Tweet Sentiments")]:
    result = (tweet_df
              .assign(**{"date": tweet_df.pd_datetime.dt.date})
              .pivot_table(index="date",
                           columns="vendor",
                           values=val,
                           aggfunc=aggfunc)
              .fillna(0))
    result = result.set_index(pd.to_datetime(result.index)).resample('M').sum()
    result.plot(ax=axs[plt_index])
    axs[plt_index].set_title(chart_name)
    axs[plt_index].set_ylabel(chart_ylabel)
plt.show()